In [2]:
%load_ext autoreload
%autoreload 2
import pandas as pd
from utils import *
import matplotlib.pyplot as plt

plt.rcParams['font.family'] = 'Helvetica'

description =(
'''From the previous experiments, there are different model shapes derived.
Lets take each of these shapes and do IMP with 20 pruning levels.
'''
)

In [3]:
sweep_ids = ['p30nbq46', 'r0b16unk', 'ocfuu4ip']
try: print('Histories already loaded.') if hc else None
except: hc = load_histories_and_configs(sweep_ids)